<a href="https://colab.research.google.com/github/indiegolab/nlp_class/blob/master/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1_with_RNN(text_generation)_%ED%95%9C%EA%B8%80%ED%85%8D%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 순환 신경망을 활용한 문자열 생성

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/text/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/text/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    깃허브(GitHub) 소스 보기</a>
  </td>
</table>

이 튜토리얼에서는 문자 기반 순환 신경망(RNN, Recurrent Neural Network)을 사용하여 어떻게 텍스트를 생성하는지 알아보자.


 Andrej Karpathy의 [순환 신경망의 뛰어난 효율](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)에서 가져온 셰익스피어 데이터셋으로 작업할 것이다.

이 데이터 셋에서 문자 시퀀스 ("Shakespear")가 주어지면, 시퀀스의 다음 문자("e")를 예측하는 모델을 훈련한다. 모델을 반복하여 호출하면 더 긴 텍스트 시퀀스 생성이 가능하다.  
.

Note: 런타임 유형: GPU

이번 실습은 [tf.keras](https://www.tensorflow.org/programmers_guide/keras)와 [즉시 실행(eager execution)](https://www.tensorflow.org/programmers_guide/eager)을 활용하여 구현된 실행 가능한 코드가 포함한다.

다음은 이 튜토리얼의 30번의 에포크(Epoch)로 훈련된 모델에서 "Q" 문자열로 시작될 때의 샘플 출력이다.

문장 중 일부는 문법적으로 맞지만 대부분 자연스럽지 않다. 이 모델은 단어의 의미를 학습하지는 않았지만, 고려해야 할 점으로 다음과 같은 것들이 있다.

* 모델은 문자 기반이다. 훈련이 시작되었을 때, 이 모델은 영어 단어의 철자를 모르거나 심지어 텍스트의 단위가 단어라는 것도 모른다.

* 출력의 구조는 대본과 유사하다. 즉, 텍스트 블록은 대개 화자의 이름으로 시작하고 이 이름들은 모든 데이터셋에서 대문자로 씌여져있다.

* 아래에 설명된 것처럼 이 모델은 작은 텍스트 배치(각 100자)로 훈련되었으며 논리적인 구조를 가진 더 긴 텍스트 시퀀스를 생성할 수 있다.

## 설정

### 텐서플로와 다른 라이브러리 임포트

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from __future__ import absolute_import, division, print_function, unicode_literals


#tensorflow 2.0 임포트
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

# !pip install -q tensorflow-gpu==2.0.0-rc1

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
# tf.enable_eager_execution()

import numpy as np
import os
import time

# !pip show tensorflow

TensorFlow 2.x selected.


In [2]:
tf.__version__

'2.0.0'

In [3]:
tf.executing_eagerly()

True

### 데이터셋 다운로드

다음 코드를 실행하여 데이터를 불러온다.

### 데이터 읽기

먼저, 텍스트를 살펴보자.

In [0]:
'''
kor 한영 데이터가져와서
pandas로 가져옴-> list로 변환
-> text
'''

#그외 전처리를 위한 라이브러리
import unicodedata
import re
import numpy as np
import pandas as pd
import os
import io
import time

In [0]:
#데이터를 사용하기 위해 구글 드라이브를 마운트 한다. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# 파일 경로를 가져온다
path_to_file = "/content/drive/My Drive/kor.xlsx"

In [0]:
#전처리 함수
def preprocess_sentence(w):
    w = w.lower().strip()

    # 문장의 마침표를 공백으로 띄어주는 과정
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # 영어, 마침표, 물음표, 느낌표, 쉼표 이외의 문자나 특수문자는 전부 제외한다 <- 한글 데이터 사용 시 코드 수정
    # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    #좌 우 공백을 제거한다
    w = w.rstrip().strip()
    return w

In [0]:
# 1. 강조표현을 제거한다
# 2. 문장을 깔끔하게 다듬는다.
def create_dataset(path, num_examples):
    df=pd.read_excel(path, sheet_name='Sheet1')
    df.ko = df.ko.apply(preprocess_sentence)
    
    # 글자로 나누기
    return list(df.ko)[:num_examples]

In [0]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  
                
# Driver code     
# print(listToString(text)) 

In [0]:
text = create_dataset(path_to_file, 10000)
text = listToString(text)
# text

In [0]:
# text = "안녕하세요. 딥러닝 활용 자연어처리 책입니다. 만나서 반갑습니다."

In [0]:
#가장 긴 문장의 길이를 가져온다
def max_length(tensor):
    return max(len(t) for t in tensor)

In [127]:
max_length(text)

1

In [0]:
# dfToList = df['ko'][:1000].tolist()
# # 1000개 문장을 list로
# print("dfToList is", dfToList, "and it's a", type(dfToList))

In [129]:
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 245212자


In [130]:
# 텍스트의 처음 250자를 살펴봅니다
print(text[:250])

나는 매일 저녁 배트를 만나러 다락방으로 가요 .선생님 이문장이 이해가 안 가요 .컴퓨터를 시작하면 시간이 너무 빠르게 가요 .나는 오늘 자정에 한국으로 돌아 가요 .나는 일어나자마자 화장실에 가요 .지금 잠을 자면 깨어나지 못할 거 같아서 지금 가요 .학교가 끝나자마자 기숙사로 가요 .대한민국 남자라면 모두 20 대에 의무적으로 군대에 가요 .오늘밤에 비자 때문에 한국에 가요 .오늘은 새 자동차를 받으러 가요 .급한 일이 있어서 손님 만나러 


In [131]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 1185개


## 텍스트 처리

### 텍스트 벡터화

훈련 전, 문자들을 수치화할 필요가 있다. 두 개의 조회 테이블(lookup table)을 만든다:  
하나는 문자를 숫자에 매핑하고 다른 하나는 숫자를 문자에 매핑하는 것이다.

In [0]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

이제 각 문자에 대한 정수 표현을 만들었습니다. 문자를 0번 인덱스부터 고유 문자 길이까지 매핑한 것을 기억합시다.

In [133]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '%' :   2,
  '&' :   3,
  ',' :   4,
  '.' :   5,
  '0' :   6,
  '1' :   7,
  '2' :   8,
  '3' :   9,
  '4' :  10,
  '5' :  11,
  '6' :  12,
  '7' :  13,
  '8' :  14,
  '9' :  15,
  '?' :  16,
  '\\':  17,
  '\xa0':  18,
  'ᄏ' :  19,
  ...
}


In [134]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'나는 매일 저녁 배트를 ' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [ 165  227    0  439  798    0  822  200    0  503 1048  415    0]


### 예측 과정

주어진 문자나 문자 시퀀스가 주어졌을 때, 다음 문자로 가장 가능성 있는 문자는 무엇일까?


 이는 모델을 훈련하여 수행할 작업입니다. 모델의 입력은 문자열 시퀀스가 될 것이고, 모델을 훈련시켜 출력을 예측합니다. 이 출력은 현재 타임 스텝(time step)의 다음 문자입니다.

RNN은 이전에 본 요소에 의존하는 내부 상태를 유지하고 있으므로, 이 순간까지 계산된 모든 문자를 감안할 때, 다음 문자는 무엇일까요?

### 훈련 샘플과 타깃 만들기

다음으로 텍스트를 샘플 시퀀스로 나눕니다. 각 입력 시퀀스에는 텍스트에서 나온 `seq_length`개의 문자가 포함될 것입니다.

각 입력 시퀀스에서, 해당 타깃은 한 문자를 오른쪽으로 이동한 것을 제외하고는 동일한 길이의 텍스트를 포함합니다.

따라서 텍스트를`seq_length + 1`개의 청크(chunk)로 나눕니다. 예를 들어, `seq_length`는 4이고 텍스트를 "Hello"이라고 가정해 봅시다. 입력 시퀀스는 "Hell"이고 타깃 시퀀스는 "ello"가 됩니다.

이렇게 하기 위해 먼저 `tf.data.Dataset.from_tensor_slices` 함수를 사용해 텍스트 벡터를 문자 인덱스의 스트림으로 변환합니다.

In [135]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

나
는
 
매
일


`batch` 메서드는 이 개별 문자들을 원하는 크기의 시퀀스로 쉽게 변환할 수 있습니다.

In [136]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

# idx2cahr를 통해 index를 문자로 다시 변환했을 때 제대로 출력되는 것을 확인할 수 있다.

'나는 매일 저녁 배트를 만나러 다락방으로 가요 .선생님 이문장이 이해가 안 가요 .컴퓨터를 시작하면 시간이 너무 빠르게 가요 .나는 오늘 자정에 한국으로 돌아 가요 .나는 일어나자마'
'자 화장실에 가요 .지금 잠을 자면 깨어나지 못할 거 같아서 지금 가요 .학교가 끝나자마자 기숙사로 가요 .대한민국 남자라면 모두 20 대에 의무적으로 군대에 가요 .오늘밤에 비자 '
'때문에 한국에 가요 .오늘은 새 자동차를 받으러 가요 .급한 일이 있어서 손님 만나러 가요 .그는 조금 있으면 수원으로 가요 .동물병원도 있기 때문에 강아지들을 데리고도 자주 가요 '
'.씻고 교복을 입고 학교로 가요 .내일 아침 일찍 얼음 낚시 가요 .다른 어떤 일을 하기 위해 서울 가요 .영화 같은 일이 현실이 되어 가요 .나도 항상 일찍 일어나서 학교를 가요 '
'.나는 이제 일 마치고 집에 가요 .그녀는 목요일에 항상 피아노 학원에 가요 .그녀는 화요일에 항상 할아버지 집에 가요 .우리는 금요일마다 음악 학원에 가요 .우리는 일주일에 5일 '


각 시퀀스에서, `map` 메서드를 사용해 각 배치에 간단한 함수를 적용하고 입력 텍스트와 타깃 텍스트를 복사 및 이동합니다:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

첫 번째 샘플의 타깃 값을 출력합니다:

In [138]:
for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  '나는 매일 저녁 배트를 만나러 다락방으로 가요 .선생님 이문장이 이해가 안 가요 .컴퓨터를 시작하면 시간이 너무 빠르게 가요 .나는 오늘 자정에 한국으로 돌아 가요 .나는 일어나자'
타깃 데이터:  '는 매일 저녁 배트를 만나러 다락방으로 가요 .선생님 이문장이 이해가 안 가요 .컴퓨터를 시작하면 시간이 너무 빠르게 가요 .나는 오늘 자정에 한국으로 돌아 가요 .나는 일어나자마'


이 벡터의 각 인덱스는 하나의 타임 스텝(time step)으로 처리됩니다. 타임 스텝 0의 입력으로 모델은 "F"의 인덱스를 받고 다음 문자로 "i"의 인덱스를 예측합니다. 다음 타임 스텝에서도 같은 일을 하지만 RNN은 현재 입력 문자 외에 이전 타임 스텝의 컨텍스트**(context)**를 고려합니다.

In [139]:
# 'First' 각 문자에 대해 입력과 출력을 확인

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 165 ('나')
  예상 출력: 227 ('는')
   1단계
  입력: 227 ('는')
  예상 출력: 0 (' ')
   2단계
  입력: 0 (' ')
  예상 출력: 439 ('매')
   3단계
  입력: 439 ('매')
  예상 출력: 798 ('일')
   4단계
  입력: 798 ('일')
  예상 출력: 0 (' ')


### 훈련 배치 생성

텍스트를 다루기 쉬운 시퀀스로 분리하기 위해 `tf.data`를 사용했습니다. 그러나 이 데이터를 모델에 넣기 전에 데이터를 섞은 후 배치를 만들어야 합니다.

In [140]:
# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 모델 설계

모델을 정의하려면 `tf.keras.Sequential`을 사용합니다. 이 간단한 예제에서는 3개의 층을 사용하여 모델을 정의합니다:

* `tf.keras.layers.Embedding` : 입력층. `embedding_dim` 차원 벡터에 각 문자의 정수 코드를 매핑하는 훈련 가능한 검색 테이블.
* `tf.keras.layers.GRU` : 크기가 `units = rnn_units`인 RNN의 유형(여기서 LSTM층을 사용할 수도 있습니다.)
* `tf.keras.layers.Dense` : 크기가 `vocab_size`인 출력을 생성하는 출력층.

In [0]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [0]:
# 모델 정의

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

각 문자에 대해 모델은 임베딩을 검색하고, 임베딩을 입력으로 하여 GRU를 1개의 타임 스텝으로 실행하고, 완전연결층을 적용하여 다음 문자의 로그 가능도(log-likelihood)를 예측하는 로짓을 생성합니다:

![모델을 통과하는 데이터의 사진](https://tensorflow.org/tutorials/text/images/text_generation_training.png)

## 모델 사용

이제 모델을 실행하여 원하는대로 동작하는지 확인합니다.

먼저 출력의 형태를 확인합니다:

In [144]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 1185) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


위 예제에서 입력의 시퀀스 길이는 100이지만 모델은 임의 길이의 입력에서 실행될 수 있습니다.

In [145]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           303360    
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_5 (Dense)              (64, None, 1185)          1214625   
Total params: 6,764,961
Trainable params: 6,764,961
Non-trainable params: 0
_________________________________________________________________


모델로부터 실제 예측을 얻으려면 출력 배열에서 샘플링하여 실제 문자 인덱스를 얻어야 합니다. 이 분포는 문자 어휘에 대한 로짓에 의해 정의됩니다.

Note: 배열에 argmax를 취하면 모델이 쉽게 루프에 걸릴 수 있으므로 배열에서 샘플링하는 것이 중요합니다.

배치의 첫 번째 샘플링을 시도해 봅시다:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

이렇게 하면 각 타임 스텝(time step)에서 다음 문자 인덱스에 대한 예측을 제공합니다:

In [147]:
sampled_indices

array([ 779,  183, 1040,   61,  624,  352,  784, 1023, 1070,   13, 1029,
        815, 1020,  145,   75, 1080,  828,  406, 1015,  274,  745,  183,
       1129,   82,  170,  969,  911,  371,  553,  411,  124, 1176,  916,
        837,  670, 1011,  835,  629,  346,  975,  565,  238,  662,  340,
        699, 1161, 1104,  501,  173, 1075,  355,  752,  666,  800,  815,
       1027,  736, 1020, 1047,  127,  441,  816,  489,  460,  589,  991,
        747,  321,  119,  122,  383,  321,  205,  686,  725,  945,  624,
        206, 1122,  731,  871,  989,   53,   53,  323,  280,  229,  262,
        528,  109,  139,  279,   72,  953,  514,  996,  773,  187,  593,
        162])

훈련되지 않은 모델에 의해 예측된 텍스트를 보기 위해 복호화합니다.

In [148]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 '어 .당신은 사람을 기쁘게 하는 재능을 가지고 있어 .그곳은 음악을 들을 수도 있고 춤을 출수도 있어 .예술은 마음을 치유해 주는 힘이 있어 .러셀은 자신을 포함해서 다른 물건 들'

예측된 다음 문자: 
 '웬냇통겹쇼랑유터팽7테잤탠꾹공펭접룻탑돈옛냇허괴남캣찻럿빔률깨흡책져썰탁젤술락컷뻬님싼뜩액횡픔방났페래옵쌓잃잤텀연탠튜꺼맨장및모샐쿤옥땐깎깜렬땐노아엉충쇼녹핸엘진쿄겐겐떠동늙덜복긍꼽돗곰치범퀵웅냥샴낀'


## 모델 훈련

이 문제는 표준 분류 문제로 취급될 수 있습니다. 이전 RNN 상태와 이번 타임 스텝(time step)의 입력으로 다음 문자의 클래스를 예측합니다.

### 옵티마이저 붙이기, 그리고 손실 함수

표준 `tf.keras.losses.sparse_softmax_crossentropy` 손실 함수는 이전 차원의 예측과 교차 적용되기 때문에 이 문제에 적합합니다.

이 모델은 로짓을 반환하기 때문에 `from_logits` 플래그를 설정해야 합니다.

In [149]:
# 손실 함수 정의
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 1185)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           7.077508


`tf.keras.Model.compile` 메서드를 사용하여 훈련 절차를 설정합니다. 기본 매개변수의 `tf.keras.optimizers.Adam`과 손실 함수를 사용합니다.

In [0]:
# adam 손실 함수 사용
model.compile(optimizer='adam', loss=loss)

### 체크포인트 구성

`tf.keras.callbacks.ModelCheckpoint`를 사용하여 훈련 중 체크포인트(checkpoint)가 저장되도록 합니다:

In [0]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 훈련 실행

훈련 시간이 너무 길지 않도록 모델을 훈련하는 데 10개의 에포크(Epoch)를 사용합니다. 코랩(Colab)에서는 런타임을 GPU로 설정하여 보다 빠르게 훈련할 수 있습니다.

In [0]:
EPOCHS=30

In [153]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# 점점 Loss가 줄어듦을 알 수 있다.

Epoch 1/30
37/37 [==============================] - 10s 271ms/step - loss: 4.6957
Epoch 2/30
37/37 [==============================] - 8s 213ms/step - loss: 4.0497
Epoch 3/30
37/37 [==============================] - 8s 213ms/step - loss: 3.7132
Epoch 4/30
37/37 [==============================] - 8s 214ms/step - loss: 3.5011
Epoch 5/30
37/37 [==============================] - 8s 214ms/step - loss: 3.3312
Epoch 6/30
37/37 [==============================] - 8s 215ms/step - loss: 3.1578
Epoch 7/30
37/37 [==============================] - 8s 215ms/step - loss: 3.0064
Epoch 8/30
37/37 [==============================] - 8s 216ms/step - loss: 2.8973
Epoch 9/30
37/37 [==============================] - 8s 215ms/step - loss: 2.8088
Epoch 10/30
37/37 [==============================] - 8s 219ms/step - loss: 2.7282
Epoch 11/30
37/37 [==============================] - 8s 214ms/step - loss: 2.6592
Epoch 12/30
37/37 [==============================] - 8s 214ms/step - loss: 2.5990
Epoch 13/30
37/37 [=====

## 텍스트 생성

### 최근 체크포인트 복원

이 예측 단계를 간단히 유지하기 위해 배치 크기로 1을 사용합니다.

RNN 상태가 타임 스텝에서 타임 스텝으로 전달되는 방식이기 때문에 모델은 한 번 빌드된 고정 배치 크기만 허용합니다.

다른 배치 크기로 모델을 실행하려면 모델을 다시 빌드하고 체크포인트에서 가중치를 복원해야 합니다.

In [154]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [156]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            303360    
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_6 (Dense)              (1, None, 1185)           1214625   
Total params: 6,764,961
Trainable params: 6,764,961
Non-trainable params: 0
_________________________________________________________________


### 예측 루프

다음 코드 블록은 텍스트를 생성합니다:

* 시작 문자열 선택과 순환 신경망 상태를 초기화하고 생성할 문자 수를 설정하면 시작됩니다.

* 시작 문자열과 순환 신경망 상태를 사용하여 다음 문자의 예측 배열을 가져옵니다.

* 다음, 범주형 배열을 사용하여 예측된 문자의 인덱스를 계산합니다. 이 예측된 문자를 모델의 다음 입력으로 활용합니다.

* 모델에 의해 리턴된 RNN 상태는 모델로 피드백되어 이제는 하나의 단어가 아닌 더 많은 컨텍스트를 갖추게 됩니다. 다음 단어를 예측한 후 수정된 RNN 상태가 다시 모델로 피드백되어 이전에 예측된 단어에서 더 많은 컨텍스트를 얻으면서 학습하는 방식입니다.

![텍스트를 생성하기 위해 모델의 출력이 입력으로 피드백](https://tensorflow.org/tutorials/text/images/text_generation_sampling.png)

생성된 텍스트를 보면 모델이 언제 대문자로 나타나고, 절을 만들고 셰익스피어와 유사한 어휘를 가져오는지 알 수 있습니다. 훈련 에포크(Epoch)가 적은 관계로 논리적인 문장을 형성하는 것은 아직 훈련되지 않았습니다.

In [0]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # 온도가 높으면 더 의외의 텍스트가 됩니다.
  # 최적의 세팅을 찾기 위한 실험
  temperature = 1.0

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [162]:
print(generate_text(model, start_string=u"정말 "))

정말 폭르나오는 종류의 이 버지를 확실하고 싶어 .당신이 프로드에서 양혹을 모망할 수 있지 .초등이 큰 감이가 비프라 , 폰의 상관에 충수할 수 있어 .축구의 불량이 아니가 내고 생활하고 싶어 .나는 주사 오색 이메일을 받으니라는 할으로 돈으로 나의 이겨 나올지 않아 .우 차리상사할 때 포츠의 활익들도 예측할 수능없어 .내지 확실히 하면 내티시를 볼 수 있어 .가방은 아두에서 구집을 내용기 위해 자주 들이고 있어 .좋은 저즈국은 집중점에 갔어 진 천체 있고 건강한 것이 그 노력에 있어 .금정 들이 이뤄지 않으면 끝나길 바래 .이제부터는 아이것이 마디어트가 송절적인 게 없어 .그는 페이지에는 크게 마별을 모끼겠던 것 다니 내 출풍이 그룹긴 변내가에 갔어 .우리는 공전권 우리의 관정샘플을 통해야하고 있어 .너의 돈을 위해서 시간 같이 늘이나서 따라울인 것도록 여행을 갔어 .나의 산기객 순산은 아껴지고 있어 .육관한 사람은 얌전에 달려가 없어 .미국이 메일을 작곡을 가족하면 될까 ?나는 이상할증이 뉴는데 아람 식사망을 주고 있어 .이번주 운전체 예약을 참고했다면 당신의 계획을 위한 연락을 판매할 것 같아 .생각의 정음으로 그걸 색깔을 우리면 눌러 갔어 .우린 바라는 노래를 부분 할 수 있지 않아 .바닥 도착해주면 건거 할 수 없어 .메일로 숙소한 시험에 갈깝지 않아 .아리디린 순간은 결혼한 터 그들을 위해 준비해보는 노치를 많아 .우리는 전달해서 얼마나 안 있는 것 같아 .그렇게 존재 더워르면 난 그 그것을 금융 하게에서 일을 해주길 바래 .나는 능력 여성의 사무실을 훔래 .나는 너가 이 옷임에 걸릴것 같아 .사랑 함께 후 아디에서는 검거색 카드를 즐길 수 없어 .내 오래 .나는 부멍된 영어를 두 개의 마실장 회신과 절대등 가지 친구를 써 알고 있어 .그들은 남편과체에 진식과 시대가 있어 .그 룸였을 때 잔울을 부분없게 놀 거 같아 .할색다운 미위로 큰 별씨가 되고 있어 .완벽히라는 데리남에서 일하여 사무실을 내용할 수 없어 .지금도 그곳에 간절적으로 우리 무엇을 

결과를 개선하는 가장 쉬운 방법은 더 오래 훈련하는 것입니다(`EPOCHS = 30`을 시도해 봅시다).

또한 다른 시작 문자열을 시험해 보거나 모델의 정확도를 높이기 위해 다른 RNN 레이어를 추가하거나 온도 파라미터를 조정하여 많은 혹은 적은 임의의 예측을 생성할 수 있습니다.

## 고급: 맞춤식 훈련

위의 훈련 절차는 간단하지만 많은 권한을 부여하지는 않습니다.

이제 수동으로 모델을 실행하는 방법을 살펴 보았으니 이제 훈련 루프를 해제하고 직접 구현합시다.
이는 시작점을 제공해 주는데, 예를 들어 커리큘럼 학습(curriculum learning)을 구현하면 모델의 오픈 루프(open-loop) 출력을 안정적으로 하는 데 도움을 줍니다.

기울기 추적을 위해 `tf.GradientTape`을 사용합니다. 이 방법에 대한 자세한 내용은 [즉시 실행 가이드](https://www.tensorflow.org/guide/eager)를 참조하십시오.

절차는 다음과 같이 동작합니다:

* 먼저 RNN 상태를 초기화합니다. 우리는`tf.keras.Model.reset_states` 메서드를 호출하여 이를 수행합니다.

* 다음으로 데이터셋(배치별로)를 반복하고 각각에 연관된 *예측*을 계산합니다.

* `tf.GradientTape`를 열고 그 컨텍스트에서의 예측과 손실을 계산합니다.

* `tf.GradientTape.grads` 메서드를 사용하여 모델 변수에 대한 손실의 기울기를 계산합니다.

* 마지막으로 옵티마이저의 `tf.train.Optimizer.apply_gradients` 메서드를 사용하여 이전 단계로 이동합니다.

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# 훈련 횟수
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # 모든 에포크(Epoch)의 시작에서 은닉 상태를 초기화
  # 초기 은닉 상태는 None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = '에포크 {} 배치 {} 손실 {}'
      print(template.format(epoch+1, batch_n, loss))

  # 모든 5 에포크(Epoch)마다(체크포인트) 모델 저장
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('에포크 {} 손실 {:.4f}'.format(epoch+1, loss))
  print ('1 에포크 당 {}초 소요\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))